In [14]:
import skrub

import sempipes

sempipes.update_config(
    llm_for_code_generation=sempipes.LLM(
        name="gemini/gemini-2.5-pro",
        parameters={"temperature": 0.5},
    )
)

dataset = skrub.datasets.fetch_credit_fraud()

products = skrub.var("products", dataset["products"])
baskets = skrub.var("baskets", dataset["baskets"][["ID"]])
labels = skrub.var("fraud_flags", dataset["baskets"]["fraud_flag"])

In [16]:
products

<Var 'products'>
Result:
―――――――
        basket_ID                              item  cash_price         make  \
1           51113  COMPUTER PERIPHERALS ACCESSORIES         409        APPLE   
9           41798                         COMPUTERS        1187        APPLE   
11          39361                         COMPUTERS         898        APPLE   
15          38615  COMPUTER PERIPHERALS ACCESSORIES         379        APPLE   
16          70262                         COMPUTERS        1899        APPLE   
...           ...                               ...         ...          ...   
163352      42613                 BEDROOM FURNITURE         259  SILENTNIGHT   
163353      42613                 OUTDOOR FURNITURE         949   LG OUTDOOR   
163354      43567                         COMPUTERS        1099        APPLE   
163355      43567                         COMPUTERS        2099        APPLE   
163356      68268           TELEVISIONS HOME CINEMA         799           LG   

                                                    model goods_code  \
1       APPLE WATCH SERIES 6 GPS 44MM SPACE GREY ALUMI...  239001518   
9       2020 APPLE MACBOOK PRO 13 TOUCH BAR M1 PROCESS...  239246780   
11      2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 ...  239246776   
15      APPLE WATCH SERIES 6 GPS 40MM BLUE ALUMINIUM CASE  239001540   
16      2021 APPLE MACBOOK PRO 14 M1 PRO PROCESSOR 16G...  240575990   
...                                                   ...        ...   
163352  SILENTNIGHT SLEEP GENIUS FULL HEIGHT HEADBOARD...  236938439   
163353  LG OUTDOOR BERGEN 2-SEAT GARDEN SIDE TABLE REC...  239742814   
163354  2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-F...  240040978   
163355  2020 APPLE IMAC 27 ALL-IN-ONE INTEL CORE I7 8G...  238923518   
163356  LG OLED48A16LA 2021 OLED HDR 4K ULTRA HD SMART TV  239866717   

        Nbr_of_prod_purchas  
1                         1  
9                         1  
11                        1  
15                        1  
16                        1  
...                     ...  
163352                    1  
163353                    1  
163354                    1  
163355                    1  
163356                    1  

[109380 rows x 7 columns]

In [17]:
aggregated = baskets.sem_agg_features(
    products,
    left_on="ID",
    right_on="basket_ID",
    nl_prompt="""
        Generate product features that are indicative of potentially fraudulent baskets, make it easy to distinguish
        anomalous baskets from regular ones! It might be helpful to combine different product statistics.
        """,
    name="basket_features",
    how_many=10,
)

aggregated

--- Fitting sempipes.sem_agg_features('Generate product features that...', 10) on dataframes of shape (61241, 1) and (109380, 7) in mode 'preview'.
	> Querying 'gemini/gemini-2.5-pro' with 2 messages...'
	> Computed 10 new feature columns: ['basket_total_value', 'basket_avg_item_value', 'basket_max_item_value', 'basket_item_value_std', 'basket_total_item_quantity', 'basket_item_line_count', 'basket_unique_category_count', 'basket_unique_make_count', 'basket_zero_price_item_count', 'basket_apple_value_ratio'].


<basket_features | Apply LLMCodeGenSemAggFeaturesEstimator>
Result:
―――――――
          ID  basket_total_value  basket_avg_item_value  \
0      51113                 409             409.000000   
1      41798                1187            1187.000000   
2      39361                 898             898.000000   
3      38615                 379             379.000000   
4      70262                1899            1899.000000   
...      ...                 ...                    ...   
61236  21243                 306             153.000000   
61237  45891                 898             898.000000   
61238  42613                1727             575.666667   
61239  43567                3198            1599.000000   
61240  68268                 799             799.000000   

       basket_max_item_value  basket_item_value_std  \
0                        409               0.000000   
1                       1187               0.000000   
2                        898               0.000000   
3                        379               0.000000   
4                       1899               0.000000   
...                      ...                    ...   
61236                    299             206.475180   
61237                    898               0.000000   
61238                    949             348.472859   
61239                   2099             707.106781   
61240                    799               0.000000   

       basket_total_item_quantity  basket_item_line_count  \
0                               1                       1   
1                               1                       1   
2                               1                       1   
3                               1                       1   
4                               1                       1   
...                           ...                     ...   
61236                           2                       2   
61237                           1                       1   
61238                           3                       3   
61239                           2                       2   
61240                           1                       1   

       basket_unique_category_count  basket_unique_make_count  \
0                                 1                         1   
1                                 1                         1   
2                                 1                         1   
3                                 1                         1   
4                                 1                         1   
...                             ...                       ...   
61236                             2                         2   
61237                             1                         1   
61238                             2                         2   
61239                             1                         1   
61240                             1                         1   

       basket_zero_price_item_count  basket_apple_value_ratio  
0                                 0                       1.0  
1                                 0                       1.0  
2                                 0                       1.0  
3                                 0                       1.0  
4                                 0                       1.0  
...                             ...                       ...  
61236                             0                       0.0  
61237                             0                       1.0  
61238                             0                       0.0  
61239                             0                       1.0  
61240                             0                       0.0  

[61241 rows x 11 columns]